In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-fr")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-fr")

In [13]:
with open('./data/es-fr/UNv1.0.es-fr.es', encoding='utf-8') as file:
    src_text = file.read()
    src_text = src_text[:1550]


In [14]:
import string
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('','',string.punctuation)        # remove the punctuations
  tokens = [(w.translate(table)) for w in tokens]  # list without punctuations (removing the punctuations)
  tokens = [word for word in tokens if word.isalpha()]   # remove non alphanumeric special charactors
  tokens = [word.lower() for word in tokens]               # convert into lowercase letters
  return tokens

In [37]:
tokenized = clean_text(src_text)
tokenized_text = ' '.join(tokenized)

In [16]:
translated = model.generate(**tokenizer(tokenized_text, return_tensors="pt", padding=True))

In [58]:
translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated][0]
translated_text

'd &apos; une convention générale sur les sixièmes sessions du gin à décembre point b de l &apos; ordre du jour provisoire et annotations à l &apos; ordre du jour provisoire y compris des suggestions pour l &apos; organisation des travaux additif Note du Secrétaire exécutif calendrier provisoire de travail proposé à l &apos; issue de consultations avec le Président lundi de décembre heures ouverture de la session intervention du Président intervention du Secrétaire général de l &apos; organisation météorologique mondiale adoption de l &apos; ordre du jour et organisation de la page la liste des tâches de la conférence des parties annotées annexe i et rapport sur les débats de l &apos; Assemblée générale concernant les changements climatiques la signature et la ratification de la convention les activités des organes du système des nations unies qui sont pertinentes pour les préparatifs de la première session de la conférence des parties les communications reçues des États et des organis

In [60]:
translated_text = translated_text.replace('&apos;', '\'')
translated_text

"d ' une convention générale sur les sixièmes sessions du gin à décembre point b de l ' ordre du jour provisoire et annotations à l ' ordre du jour provisoire y compris des suggestions pour l ' organisation des travaux additif Note du Secrétaire exécutif calendrier provisoire de travail proposé à l ' issue de consultations avec le Président lundi de décembre heures ouverture de la session intervention du Président intervention du Secrétaire général de l ' organisation météorologique mondiale adoption de l ' ordre du jour et organisation de la page la liste des tâches de la conférence des parties annotées annexe i et rapport sur les débats de l ' Assemblée générale concernant les changements climatiques la signature et la ratification de la convention les activités des organes du système des nations unies qui sont pertinentes pour les préparatifs de la première session de la conférence des parties les communications reçues des États et des organisations régionales d ' intégration économ

In [23]:
with open('translated_text.txt', 'w') as file:
    file.write(translated_text)
    

In [20]:
with open('./data/es-fr/UNv1.0.es-fr.fr', encoding='utf-8') as file:
    target_text = file.read()
    target_text = target_text[:1550]

In [21]:
target_text

"D'UNE CONVENTION-CADRE SUR LES CHANGEMENTS CLIMATIQUES\nSixième session\nGenève, 7-10 décembre 1992\nPoint 1 b) de l'ordre du jour provisoire\nORDRE DU JOUR PROVISOIRE ANNOTE, ASSORTI DE SUGGESTIONS\nCONCERNANT L'ORGANISATION DES TRAVAUX\nAdditif\nNote du Secrétaire exécutif\nProjet de calendrier des travaux proposé\naprès consultation du Président\nLundi 7 décembre\n15 heures Ouverture de la session\nDéclaration du Président\nDéclaration du Secrétaire général de l'Organisation\nmétéorologique mondiale\nAdoption de l'ordre du jour et organisation des travaux\nGE.92-63308/9749C (F)\npage 2\n- L'énumération des tâches de la Conférence des\nParties (ordre du jour provisoire annoté, annexe I)\net présenter des informations concernant :\n- Les débats de l'Assemblée générale sur les\nchangements climatiques\n- La signature et la ratification de la Convention\n- Les activités des organes du système des\nNations Unies qui ont un rapport avec les\npréparatifs de la première session de la\nConf

In [62]:
target_tokens = clean_text(target_text)
target_text = ' '.join(target_tokens)

# Create a list with the texts
texts = [target_text, translated_text]

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

vectorizer = TfidfVectorizer(max_features=2000, stop_words=stopwords.words('french'))
vectorized_texts = vectorizer.fit_transform(texts)

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(X=vectorized_texts)
similarity

array([[1.        , 0.69440149],
       [0.69440149, 1.        ]])

In [69]:
similarity[0][1]

0.6944014901497653

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import string



def clean_text(doc):
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

def translate_and_compare(src_text, target_text):
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-fr")
    model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-fr")

    tokenized = clean_text(src_text)
    tokenized_text = ' '.join(tokenized)

    translated = model.generate(**tokenizer(tokenized_text, return_tensors="pt", padding=True))

    translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated][0]
    translated_text = translated_text.replace('&apos;', '\'')

    with open('translated_text.txt', 'w') as file:
        file.write(translated_text)

    target_tokens = clean_text(target_text)
    target_text = ' '.join(target_tokens)

    # Create a list with the texts
    texts = [target_text, translated_text]

    vectorizer = TfidfVectorizer(max_features=2000, stop_words=stopwords.words('french'))
    vectorized_texts = vectorizer.fit_transform(texts)

    similarity = cosine_similarity(X=vectorized_texts)
    return similarity[0, 1]

# Example of iterating the process 10 times
num_iterations = 10
total_similarity = 0

for _ in range(num_iterations):
    with open('./data/es-fr/UNv1.0.es-fr.es', encoding='utf-8') as file:
        src_text = file.read()[:1500]

    with open('./data/es-fr/UNv1.0.es-fr.fr', encoding='utf-8') as file:
        target_text = file.read()[:1500]

    iteration_similarity = translate_and_compare(src_text, target_text)
    total_similarity += iteration_similarity

average_similarity = total_similarity/num_iterations

In [4]:
average_similarity

0.6718157662906455